In [1]:
from distributed import Client, LocalCluster
import click
from daskqueue.Protocol import Message
from time import perf_counter

from daskqueue import ConsumerPool, Durability, QueuePool
import tempfile

no_func = lambda: None

In [2]:
cprint = click.echo
gprint = lambda s: click.style(s, fg="green")
func = lambda x: x + 2

def rdx_msg():
    msg = Message(func, 12)
    return msg


In [5]:
cluster= LocalCluster(
    n_workers=6,
    threads_per_worker=1,
    dashboard_address=":3338",
    memory_limit="2GB"
)

2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-64juugzz', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jxt3gv4c', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_4rzcgcc', purging
2023-01-02 18:41:10,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-l335dt48', purging
2023-01-02 18:41:10,412 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cs8k9qde', purging


In [3]:
client = Client("tcp://192.168.1.181:8786",direct_to_workers=True)

/home/amine/Documents/coding/daskqueue/env/lib/python3.10/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| python  | 3.10.8.final.0 | 3.10.9.final.0 | 3.10.9.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [7]:
client.restart()

<Client: 'tcp://192.168.1.181:8786' processes=0 threads=0, memory=0 B>

In [8]:
N = 10_000
n_queues = 1
n_consumers = 1

with tempfile.TemporaryDirectory() as tmpdirname:
    s = perf_counter()
    queue_pool = QueuePool(
        client, n_queues, durability=Durability.DURABLE, dirpath=str(tmpdirname)
    )

    _ = queue_pool.batch_submit([(no_func,) for _ in range(N)])
    e = perf_counter()
    bs_ops = N / (e - s)

cprint(f"Write ops " + gprint(f"{bs_ops:.2f} wop/s"))

2023-01-03 10:52:42,485,485 INFO: Created 1 queues in Cluster and one QueueManager.


Write ops 6234.22 wop/s


In [9]:
consumer_pool = ConsumerPool(client, queue_pool, n_consumers=n_consumers)

s = perf_counter()
consumer_pool.start()
consumer_pool.join(timestep=0.1, progress=False)
consumer_pool.results()

e = perf_counter()

rps = N / (e - s)  # op/s

cprint(f"Read ops " + gprint(f"{bs_ops:.2f} r_op/s"))

2023-01-03 10:52:45,407,407 INFO: Starting 1 consumers
2023-01-03 10:52:45,408,408 INFO: Waiting for the 1 consumers to process all items in queue_pool...
2023-01-03 10:52:50,093,93 INFO: All consumers are done ! 10000 items processed. 
2023-01-03 10:52:50,094,94 INFO: Cancelling 1 consumers.


Read ops 6234.22 r_op/s


2023-01-03 11:14:45,083 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


## Test Counter Actor

In [12]:
N = 1_000_000

In [14]:
class Counter:
    n = 0

    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1
        return self.n
    def add(self, x):
        self.n += x
        return self.n

future = client.submit(Counter, actor=True)  # Create a Counter on a worker
counter = future.result()  


In [15]:
s = perf_counter()
for _ in range(N):
    future = counter.increment()                 # Call remote method
    # res = future.result() 
e = perf_counter()
wops = N / (e - s)

In [9]:
import random
while True:
    try : 
        x = random.randint(0,100)
        print(x)
        if x < 40:
            raise ValueError("error")
    except ValueError:
        break
j

41
51
38


In [10]:
import itertools
from typing import Generator, Iterable
from daskqueue.Protocol import Message

def msg_grouper(n: int, iterable: Iterable, **kwargs) -> Generator:
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield [Message(func, *args, **kwargs) for func, *args in chunk]


In [34]:
batch_size = 10
f = lambda: None
N = 100000
list_calls = [(f,) for _ in range(N)]
s = 0
for msgs in msg_grouper(10000, list_calls):
    s +=len(msgs)

In [35]:
s

100000